# Domino's API
### In this example, we'll use Domino's web API to find a local store, check its menu, and order some pizza. 

Thanks to [RIAEvangelist](https://github.com/RIAEvangelist) for compiling all of this information into a [node tool](https://github.com/RIAEvangelist/node-dominos-pizza-api) for ordering pizza.
    

In [ ]:
import requests  # this will do all the HTTPing for us
import json
import xmltodict
import time


# this is a popular data sciencing library - we're just using this to print the menu in a format we can read
import pandas as pd 

# see the README or look at the settings.py file for more information.
# if you want to use this, you'll need to enter your information into this.
# !! ONLY DO THAT IF YOU'RE RUNNING A LOCAL COPY OF JUPYTER !!
# !! DO NOT USE YOUR REAL INFO ON MYBINDER !!
from settings import DEFINITELY_NOT_MY_CREDIT_CARD, MY_PERSONAL_INFO

# !! SERIOUSLY! READ THE WARNING ABOVE !!

In [ ]:
# Little helper function for pretty-printing JSON responses
# this is not necessary but useful for demonstration purposes
import pprint as PP
pp = PP.PrettyPrinter(indent=2)
pprint = pp.pprint

In [ ]:
# API urls
BASE_URL = 'https://order.dominos.com/power/'  # !!!! use HTTPS -- you'll be sending CC info over this
STORE_FINDER_URL = BASE_URL + 'store-locator'
STORE_INFO_URL = BASE_URL + 'store/{store_id}/profile'
STORE_MENU_URL = BASE_URL + 'store/{store_id}/menu'

PRICE_ORDER_URL = BASE_URL + 'price-order'
PLACE_ORDER_URL = BASE_URL + 'place-order'
VALIDATE_ORDER_URL = BASE_URL + 'validate-order'

In [ ]:
# Before we begin, we need to have our own information to send to domino's.  So they can find us.
me = MY_PERSONAL_INFO
pprint(me)

In [ ]:
# The store-locator endpoint takes 3 parameters
# `s`: address part 1
# `c`: address part 2
# `t`: type ('Delivery' | 'Carryout')

# we just need the first address part
my_params = {
    's': me['address'],
    't': 'Delivery'
}

r = requests.get(STORE_FINDER_URL, params=my_params)

In [ ]:
response_data = r.json()
pprint(response_data)

In [ ]:
# That gives us a lot of stores so let's just look at the first(closest) one.
my_store = response_data['Stores'][0]
pprint(my_store)

In [ ]:
# Looking through that information, this seems like the right store.  We'll use its `StoreID` going forward.
store_id = my_store['StoreID']
store_id

In [ ]:
r = requests.get(STORE_MENU_URL.format(store_id=store_id))
menu = r.json()
pprint(menu)

In [ ]:
data = menu['Products']['Data']
data[0]

In [ ]:
# There's a crap-load of items in that menu, so let's filter it -- and use pandas to make it easier to read

# The `Products` table headings from Domino's JSON response
headings = menu['Products']['Columns']

# The `Products` table data
data = menu['Products']['Data']

# Put it all into a pandas dataframe
products = pd.DataFrame(data, columns=headings)

# and filter where the Category is 'Pizza'
products

In [ ]:
# The `Options` table headings from Domino's JSON response
headings = menu['Options']['Columns']

# The `Options` table data
data = menu['Options']['Data']

# Put it all into a pandas dataframe
options = pd.DataFrame(data, columns=headings)

# and filter where the Category is 'Pizza'
options[options['CategoryCode'] == 'Pizza']

In [ ]:

my_order = {
    # My info
    'FirstName': me['firstName'],
    'LastName': me['lastName'],
    'Email': me['email'],
    'Phone': me['phone'],
     'Address' : {
        'Street': '3343 Forbes Ave',
        'City': 'Pittsburgh',
        'Region': 'PA',
        'PostalCode': '15213'
    },
    #Store info
    'StoreID': store_id,
    'ServiceMethod': 'Delivery',
    
    # Products
    'Products': [
        {
            'Code':'P12IRECZ',
            'Options': {},
            'Quantity': 1
        }
    ]
}

r = requests.post(VALIDATE_ORDER_URL, json={'Order': {**my_order}})
pprint(r.json())

In [ ]:
r = requests.post(PRICE_ORDER_URL, json={'Order': my_order})
price_data = r.json()
price_data
price = price_data['Order']['Amounts']['Customer']
price

In [ ]:
DEFINITELY_NOT_MY_CREDIT_CARD['Amount'] = price
DEFINITELY_NOT_MY_CREDIT_CARD['Type'] = 'CreditCard'

In [ ]:
my_order['Payments'] = [DEFINITELY_NOT_MY_CREDIT_CARD]
r = requests.post(VALIDATE_ORDER_URL, json={'Order': my_order})
test_validation = r.json()
del test_validation['Order']['Payments']

pprint(test_validation)

In [ ]:
#r = requests.post(PLACE_ORDER_URL, json={'Order': my_order})
order_response = r.json()
order_id = order_response['Order']['OrderID']
del order_response['Order']['Payments']
order_response

In [ ]:
r = requests.get('https://trkweb.dominos.com/orderstorage/GetTrackerData', params={'Phone': me['phone']})
x = xmltodict.parse(r.text)
pprint(x)

In [ ]:
r = requests.get('https://trkweb.dominos.com/orderstorage/GetTrackerData', params={'Phone': me['phone']})
x = xmltodict.parse(r.text)
data = x['soap:Envelope']['soap:Body']['GetTrackerDataResponse']['OrderStatuses']['OrderStatus'][0]
data

In [ ]:
# These functions will help us track the order as it goes through Domino's system.

def add_time(time, emoji):
    '''Generates a line segment `time`//60 minutes long.'''
    result = ''
    if time:
        result += emoji
        for i in range(int(time) // 60):
            result += '='
    return result
        
def show_status(data):
    '''Generates and prints a ASCII art timeline of our delivery's progress'''
    result = ''
    done = False
    for stuff in [('MakeTimeSecs', '📱'), 
                  ('OvenTimeSecs', '🔥'), 
                  ('RackTimeSecs', '♨️'), 
                  ('OrderDeliveryTimeSecs', '🚘')]:
        key, emoji = stuff
        result += add_time(data[key], emoji)
    if data['OrderStatus'] == 'Complete':
        result += '😊'
        done=True
    print(result)
    return done

while not show_status(data):    # print our tracker and return `True` if the pizza is here.
    # if it's not here, we wait 3 seconds
    time.sleep(3)
    
    # and then update the tracking info
    r = requests.get('https://trkweb.dominos.com/orderstorage/GetTrackerData', params={'Phone': me['phone']})
    x = xmltodict.parse(r.text)
    data = x['soap:Envelope']['soap:Body']['GetTrackerDataResponse']['OrderStatuses']['OrderStatus'][0]
